# Setup 
Python Version: 3.13.2

Possible pip installs needed: networkx matplotlib

In [34]:
import gzip
import networkx as nx
import re
import copy
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

## Load Data From File into Graph 
Assigned: Isabell

### Load Graph

#### Data format:

Id:   15 \
ASIN: 1559362022\
title: Wake Up and Smell the Coffee\
  &emsp;group: Book\
  &emsp;salesrank: 518927\
  &emsp;similar: 5  1559360968  1559361247  1559360828  1559361018  0743214552\
  &emsp;categories: 3\
   &emsp;&emsp;|Books[283155]|Subjects[1000]|Literature & Fiction[17]|Drama[2159]|United States[2160]\
   &emsp;&emsp;|Books[283155]|Subjects[1000]|Arts & Photography[1]|Performing Arts[521000]|Theater[2154]|General[2218]\
   &emsp;&emsp;|Books[283155]|Subjects[1000]|Literature & Fiction[17]|Authors, A-Z[70021]|( B )[70023]|Bogosian, Eric[70116]\
  &emsp;reviews: total: 8  downloaded: 8  avg rating: 4\
    &emsp;&emsp;2002-5-13  cutomer: A2IGOA66Y6O8TQ  rating: 5  votes:   3  helpful:   2\
    &emsp;&emsp;2002-6-17  cutomer: A2OIN4AUH84KNE  rating: 5  votes:   2  helpful:   1\
    &emsp;&emsp;2003-1-2  cutomer: A2HN382JNT1CIU  rating: 1  votes:   6  helpful:   1\
    &emsp;&emsp;2003-6-7  cutomer: A2FDJ79LDU4O18  rating: 4  votes:   1  helpful:   1\
    &emsp;&emsp;2003-6-27  cutomer: A39QMV9ZKRJXO5  rating: 4  votes:   1  helpful:   1\
    &emsp;&emsp;2004-2-17  cutomer:  AUUVMSTQ1TXDI  rating: 1  votes:   2  helpful:   0\
    &emsp;&emsp;2004-2-24  cutomer: A2C5K0QTLL9UAT  rating: 5  votes:   2  helpful:   2\
    &emsp;&emsp;2004-10-13  cutomer:  A5XYF0Z3UH4HB  rating: 5  votes:   1  helpful:   1\

#### Graph Loading:

Graph Specifications:\
&emsp;Nodes = ASIN (Amazon Standard Identification Number)\
&emsp;Edges = similar (ASINs of co-purchased products (people who buy X also buy Y))\
&emsp;Attributes:\
    &emsp;&emsp;title (Name/title of the product)\
    &emsp;&emsp;group (Product group (Book, DVD, Video or Music))\
    &emsp;&emsp;categories (Location in product category hierarchy to which the product belongs (separated by |, category id in []))\

In [42]:
'''

    parameters:
        path: path to amazon-meta.txt.gz file that should be stored locally. can be downloaded from https://snap.stanford.edu/data/amazon-meta.html
        
    Graph Specifications:
        Nodes = ASIN (Amazon Standard Identification Number)
        Edges = similar (ASINs of co-purchased products (people who buy X also buy Y))
        Attributes:
            title (Name/title of the product)
            group (Product group (Book, DVD, Video or Music))
            categories (Location in product category hierarchy to which the product belongs (separated by |, category id in []))
            
    Note: Uses ASIN as node ids
'''
def load_amazon_undirected(path):
    G = nx.Graph()
    
    # Use regex to see if line of file is related :
    # ASIN (Amazon Standard Identification Number)
    asin_pattern = re.compile(r'^ASIN:\s+(.+)$')
    # similar (ASINs of co-purchased products)
    similar_pattern = re.compile(r'^similar:\s+(\d+)\s+(.+)$')
    # Group (Product group (Book, DVD, Video or Music))
    group_pattern = re.compile(r'^group:\s+(.+)$')
    # Title (Name/title of the product)
    title_pattern = re.compile(r'^title:\s+(.+)$')
    # Categories (Location in product category hierarchy to which the product belongs (separated by |, category id in [])) 
    categories_count_pattern = re.compile(r'^categories:\s+(\d+)$')
    
    # Keep track of current node
    current_asin = None
    current_title = None
    current_group = None
    current_categories = []
    

    # Open .txt.gz file as .txt
    with gzip.open(path, 'rt', encoding='latin-1') as f:
        # loop through each line
        for line in f:
            # Remove white space around characters
            line = line.strip()

            # New Node:     Check if line contains a new node id (aka see if line contains ASIN regex)
            if asin_match := asin_pattern.match(line):
                # If there was a last node -> assign the attribute values
                if current_asin is not None:
                    current_data = G.nodes[current_asin]
                    if current_title is not None:
                        current_data["title"] = current_title
                    if current_group is not None:
                        current_data["group"] = current_group
                    if current_categories:
                        current_data["categories"] = current_categories
                # Update current node's ID
                current_asin = asin_match.group(1)
                # Create new node
                G.add_node(current_asin)
                # Reset attribute variables
                current_title = None
                current_group = None
                current_categories = []
                
            # Node Edges and Attributes
            elif current_asin is not None:
                # EDGES/Co-Purchases:     Check if line contains node's co-purchases (aka see if line contains similar regex)
                if sim_match := similar_pattern.match(line):
                    # Grab the list of similars (aka group 2)
                    similars_str = sim_match.group(2)
                    # Split string into a list so each co-purchase's ASIN is alone
                    similars = similars_str.split()
                    # Add each 
                    for s in similars:
                        if s != 'null':
                            # Add co-purchase node (networkx does handle duplicates)
                            G.add_node(s)
                            # Create undirected edge between current node and co-purchase node
                            G.add_edge(current_asin, s)
                        
                        
                # GRAB ATTRIBUTES:
                # Title:
                elif title_match := title_pattern.match(line):
                    current_title = title_match.group(1)
                # Group:
                elif group_match := group_pattern.match(line):
                    current_group = group_match.group(1)
                # Categories: 
                elif categories_count_match := categories_count_pattern.match(line):
                    num_categories = int(categories_count_match.group(1))
                    for _ in range(num_categories):
                        cat_line = next(f).strip()
                        categs = [c for c in cat_line.split('|') if c]
                        for categ in categs:
                            if c := re.match(r'(.+)\[(\d+)\]$', categ):
                                c_name = c.group(1).strip()
                                c_id = int(c.group(2))
                                c_all = (c_name, c_id)
                                if c_all not in current_categories:
                                    current_categories.append(c_all)
    return G

def preprocess_largest_component(G):
    # Create Copy of G for preprocessing
    G_preprocessed = copy.deepcopy(G)
    
    # Duplicates are handled by networkx itself
    
    # Remove self loops
    self_loops = list(nx.selfloop_edges(G_preprocessed))
    G_preprocessed.remove_edges_from(self_loops)
    
    # Remove isolated nodes
    isolates = list(nx.isolates(G_preprocessed))
    G_preprocessed.remove_nodes_from(isolates)
    
    # Find largest connected component
    # Find all components
    comps = list(nx.connected_components(G_preprocessed))
    # Grab largest component
    largest_comp_nodes = max(comps, key=len)
    # Create subgraph of largest component and make sure its undirected
    G_Largest = G_preprocessed.subgraph(largest_comp_nodes).copy().to_undirected()
    
    return G_preprocessed, G_Largest
    
# WARNING will remove all nodes if given attribute is not in dataset at all
def remove_nodes_without_attribute(G, attribute, preprocess=True):
    # Create Copy of G for preprocessing
    G_reduced = copy.deepcopy(G)
    # Grab node set to remove
    nodes_to_remove = [
        n for n, data in G.nodes(data=True) if attribute not in data
    ]
    # Remove the set of nodes
    G_reduced.remove_nodes_from(nodes_to_remove)
    if preprocess:
        G_reduced_process, G_reduced_large = preprocess_largest_component(G_reduced)
        return G_reduced, G_reduced_process, G_reduced_large
    return G_reduced

path = './amazon-meta.txt.gz'
# Create Base graph with all information
G = load_amazon_undirected(path)
# A preprocessed Graph of G, and grab its largest component
G_processed, G_largest =  preprocess_largest_component(G)
# Create graphs, and processed graphs without 'group' attribute from largest connected component of Graph
G_group, G_group_processed, G_group_largest = remove_nodes_without_attribute(G_largest, "group")

### Graph Details:

In [41]:
def print_graph_details(G, title=None):
    if title:
        print(title)
    print(f"\t|V| (Nodes): {G.number_of_nodes()}")
    print(f"\t|E| (Edges): {G.number_of_edges()}")
    print(f"\tDensity: {nx.density(G)}")
    is_directed = G.is_directed()
    # Find largest component
    if is_directed:
        comp = list(nx.weakly_connected_components(G))
    else:
        comp = list(nx.connected_components(G))
    num_components = len(comp)
    print(f"\tIs Directed: {is_directed}")
    print(f"\tNumber of Components: {num_components}")
    print("\tDisplay first 5 nodes and their data:")
    for n, data in list(G.nodes(data=True))[:5]:
        print(f"\t\tNode {n} →")
        print(f"\t\t\t Degree → {G.degree[n]}")
        for d in data:
            print(f"\t\t\t {d} → {data.get(d)}")
        
print_graph_details(G, title="All Amazon Graph Data")
print_graph_details(G_processed, title="Preprocessed Graph (Remove Self Loops and Isolated Nodes)")
print_graph_details(G_largest, title="Largest Connected Component")
print_graph_details(G_group_largest, title="Largest Connected Component with 'group' attributes")

All Amazon Graph Data
	|V| (Nodes): 721342
	|E| (Edges): 1545228
	Density: 5.939374739244349e-06
	Is Directed: False
	Number of Components: 172167
	Display first 5 nodes and their data:
		Node 0771044445 →
			 Degree → 0
		Node 0827229534 →
			 Degree → 8
			 title → Patterns of Preaching: A Sermon Sampler
			 group → Book
			 categories → ['Books', 'Subjects', 'Religion & Spirituality', 'Christianity', 'Clergy', 'Preaching', 'Sermons']
		Node 0804215715 →
			 Degree → 31
			 title → Witness of Preaching
			 group → Book
			 categories → ['Books', 'Subjects', 'Religion & Spirituality', 'Christianity', 'Clergy', 'Preaching', 'Other Practices', 'Ritual']
		Node 156101074X →
			 Degree → 27
			 title → The Preaching Life
			 group → Book
			 categories → ['Books', 'Subjects', 'Religion & Spirituality', 'Christianity', 'Clergy', 'Preaching', 'Sermons', 'Authors, A-Z', '( T )', 'Taylor, Barbara Brown', 'Protestantism', 'Episcopalian', 'General']
		Node 0687023955 →
			 Degree → 18
			 title

### Base Graph Visualizations

using https://lite.gephi.org/v1.0.2/ to visualize graph due to large size

In [43]:
# Export a to GEXF (To use with Gephi)
def export_to_Gephi(G, name):
    # Create Copy of G to avoid changing graph
    G_copy = copy.deepcopy(G)
    # Drop categories because it is not gexf safe type
    for n, data in G_copy.nodes(data=True):
        if 'categories' in data:
            del data['categories']
    nx.write_gexf(G_copy, name+".gexf")
    
export_to_Gephi(G_group_largest, "amazon_largest_group_largest")

## Centralities
Assigned: Isabell

Questions Proposed:
1. Which products have central positions within co-purchased products?
    - Is there a correlation/connection among central products’ attributes


## Community Detection
Assigned: 

## Link Prediction
Assigned: 